In [88]:
import sympy as smp
from sympy import I, Add, Mul, N, cos, sin, simplify
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.dagger import Dagger

In [34]:
t = smp.symbols('t', real=True)

sig_i = smp.eye(2)
sig_x = smp.Matrix([[0, 1], [1, 0]])
sig_y = smp.Matrix([[0, -I], [I, 0]])
sig_z = smp.Matrix([[1, 0], [0, -1]])

In [35]:
def pauli_on_qubit(pauli_matrix, qubit_index, num_of_qubits):
    operators = [sig_i] * num_of_qubits
    operators[qubit_index] = pauli_matrix
    result = operators[0]
    for op in operators[1:]:
        result = TensorProduct(result, op)
    return result

In [36]:
def create_hamiltonian(num_of_qubits, gammas, distances):
    dim = 2 ** num_of_qubits
    hamiltonian = smp.zeros(dim, dim)
    for i in range(num_of_qubits):
        for j in range (i+1, num_of_qubits):
            if (gammas[i] == gammas[j]): # like spins
                h_int = gammas[i]*gammas[j]*(2*pauli_on_qubit(sig_z, i, num_of_qubits)*pauli_on_qubit(sig_z, j, num_of_qubits) - pauli_on_qubit(sig_x, i, num_of_qubits)*pauli_on_qubit(sig_x, j, num_of_qubits) - pauli_on_qubit(sig_y, i, num_of_qubits)*pauli_on_qubit(sig_y, j, num_of_qubits))/(4*distances[i][j]**3)
            else: # unlike spins
                h_int = gammas[i]*gammas[j]*pauli_on_qubit(sig_z, i, num_of_qubits)*pauli_on_qubit(sig_z, j, num_of_qubits)/(2*distances[i][j]**3)
            hamiltonian += h_int
    return hamiltonian

In [37]:
rho_x_polarized = smp.Rational(1, 2) * (sig_i + sig_x)
rho_y_polarized = smp.Rational(1, 2) * (sig_i + sig_y)
rho_z_polarized = smp.Rational(1, 2) * (sig_i + sig_z)
rho_u_polarized = smp.Rational(1, 2) * (sig_i)

def create_initial_density_matrix(polarization, num_of_qubits):
    if polarization[0] == 'x':
        rho = rho_x_polarized
    elif polarization[0] == 'y':
        rho = rho_y_polarized
    elif polarization[0] == 'z':
        rho = rho_z_polarized
    else:
        rho = rho_u_polarized

    for i in range(1, num_of_qubits):
        if polarization[i] == 'x':
            rho = TensorProduct(rho, rho_x_polarized)
        elif polarization[i] == 'y':
            rho = TensorProduct(rho, rho_y_polarized)
        elif polarization[i] == 'z':
            rho = TensorProduct(rho, rho_z_polarized)
        else:
            rho = TensorProduct(rho, rho_u_polarized)
    return rho

In [38]:
# Partial trace function: ChatGPT - decommissioned
def partial_trace(rho, trace_qubit, num_of_qubits):
    """
    Perform the partial trace on the given density matrix `rho` over the specified qubit `trace_qubit`.

    Parameters:
    - rho: sympy.Matrix, the density matrix.
    - trace_qubit: int, the index of the qubit to trace out (0-based).
    - num_of_qubits: int, the total number of qubits in the system.

    Returns:
    - sympy.Matrix, the reduced density matrix.
    """
    dim = 2 ** num_of_qubits
    step = 2 ** (num_of_qubits - trace_qubit - 1)
    reduced_dim = 2 ** (num_of_qubits - 1)

    reshaped_rho = smp.zeros(reduced_dim, reduced_dim)

    for i in range(reduced_dim):
        for j in range(reduced_dim):
            block_start_row = (i // step) * (2 * step) + (i % step)
            block_start_col = (j // step) * (2 * step) + (j % step)

            reshaped_rho[i, j] = sum(
                rho[block_start_row + k * step, block_start_col + k * step]
                for k in range(2)
            )

    return reshaped_rho


In [39]:
def partial_trace_multiple(rho, trace_qubits, num_of_qubits):
    """
    Perform the partial trace on the given density matrix `rho` over multiple specified qubits.

    Parameters:
    - rho: sympy.Matrix, the density matrix.
    - trace_qubits: list of int, the indices of the qubits to trace out (0-based).
    - num_of_qubits: int, the total number of qubits in the system.

    Returns:
    - sympy.Matrix, the reduced density matrix.
    """
    remaining_qubits = sorted(set(range(num_of_qubits)) - set(trace_qubits))
    num_remaining_qubits = len(remaining_qubits)
    reduced_dim = 2 ** num_remaining_qubits
    
    # Initialize the reduced density matrix
    reduced_rho = smp.zeros(reduced_dim, reduced_dim)
    
    # Map indices in the reduced space to indices in the full space
    for i in range(reduced_dim):
        for j in range(reduced_dim):
            full_indices_i = list(bin(i)[2:].zfill(num_remaining_qubits))
            full_indices_j = list(bin(j)[2:].zfill(num_remaining_qubits))
            
            trace_sum = 0
            for traced_bits in range(2 ** len(trace_qubits)):
                trace_indices = list(bin(traced_bits)[2:].zfill(len(trace_qubits)))
                
                full_i = [None] * num_of_qubits
                full_j = [None] * num_of_qubits
                
                for qi, ri in enumerate(remaining_qubits):
                    full_i[ri] = full_indices_i[qi]
                    full_j[ri] = full_indices_j[qi]
                
                for ti, qi in enumerate(trace_qubits):
                    full_i[qi] = trace_indices[ti]
                    full_j[qi] = trace_indices[ti]
                
                full_i_idx = int(''.join(full_i), 2)
                full_j_idx = int(''.join(full_j), 2)
                
                trace_sum += rho[full_i_idx, full_j_idx]
            
            reduced_rho[i, j] = trace_sum
    
    return reduced_rho

In [40]:
def create_evolve_expect(gammas, distances, polarization, num_of_qubits):
    # create
    hamiltonian = create_hamiltonian(num_of_qubits, gammas, distances)
    rho_total_0 = create_initial_density_matrix(polarization, num_of_qubits)
    
    # evolve
    U = smp.exp(-I * hamiltonian * t)
    rho_total_t = U * rho_total_0 * Dagger(U)
    rho_0_t = partial_trace_multiple(rho_total_t, [i for i in range (1, num_of_qubits)], num_of_qubits)

    # print(hamiltonian)
    # print(rho_total_0)
    # print(rho_0_t)

    # for i in range(1, num_of_qubits):
    #     rho_t = partial_trace(rho_t, i, num_of_qubits)

    # expect
    expectation_Sx = smp.trace(sig_x * rho_0_t).simplify()
    expectation_Sy = smp.trace(sig_y * rho_0_t).simplify()
    expectation_Sz = smp.trace(sig_z * rho_0_t).simplify()
    return (expectation_Sx, expectation_Sy, expectation_Sz)


In [46]:
# Define a function to ignore terms smaller than a threshold
def filter_small_terms(expr, threshold=1e-3):
    if expr.is_Atom:
        # For atomic values (numbers, symbols), check their magnitude
        if abs(expr) < threshold:
            return 0
        else:
            return expr
    elif expr.is_Add:
        # Process addition terms
        return Add(*[filter_small_terms(arg, threshold) for arg in expr.args])
    elif expr.is_Mul:
        # Process multiplication terms
        coeff, factors = expr.as_coeff_mul()
        if abs(coeff) < threshold:
            return 0
        return Mul(coeff, *[filter_small_terms(f, threshold) for f in factors])
    else:
        return expr

In [110]:
# 2 qubits
# gammas = [0.1, 0.1]
# distances = [["NA", 1], [1, "NA"]]
# polarization = ['x','u']
# num_of_qubits = 2

# 3 qubits
gammas = [0.1, 0.1, 0.02]
distances = [["NA", 1, 1], [1, "NA", 1], [1, 1, "NA"]]
polarization = ['y','z','z']
num_of_qubits = 3

expression = create_evolve_expect(gammas, distances, polarization, num_of_qubits)
filtered_exp = tuple(filter_small_terms(e) for e in expression)
rounded_num = tuple(N(e, 2) for e in filtered_exp)
rounded_num_sincos = tuple(simplify(e.rewrite(sin)) for e in rounded_num)
print(rounded_num_sincos)
# print(smp.latex(rounded_num_sincos))

(-0.25*sin(0.007*t) - 0.25*sin(0.007*t) - 0.5*sin(0.017*t) + 0.25*I*cos(0.007*t) - 0.25*I*cos(0.007*t), 0.25*exp(0.007*I*t) + 0.25*exp(0.017*I*t) + 0.25*exp(-0.017*I*t) + 0.25*exp(-0.007*I*t), 0.5 - 0.5*cos(0.01*t))
\left( - 0.25 \sin{\left(0.007 t \right)} - 0.25 \sin{\left(0.007 t \right)} - 0.5 \sin{\left(0.017 t \right)} + 0.25 i \cos{\left(0.007 t \right)} - 0.25 i \cos{\left(0.007 t \right)}, \  0.25 e^{0.007 i t} + 0.25 e^{0.017 i t} + 0.25 e^{- 0.017 i t} + 0.25 e^{- 0.007 i t}, \  0.5 - 0.5 \cos{\left(0.01 t \right)}\right)


In [106]:
# import itertools

In [111]:
# # Define the possible values
# values = ['u', 'x', 'y', 'z']

# # Length of the array (number of positions)
# array_length = 3  # Change this to your desired length

# # Generate all combinations
# combinations = list(itertools.product(values, repeat=array_length))

# # Print each combination
# for combo in combinations:
#     expression = create_evolve_expect(gammas, distances, combo, num_of_qubits)
#     filtered_exp = tuple(filter_small_terms(e) for e in expression)
#     rounded_num = tuple(N(e, 2) for e in filtered_exp)
#     rounded_num_sincos = tuple(simplify(e.rewrite(sin)) for e in rounded_num)
#     print(f"{combo} {rounded_num_sincos}")
#     # print(smp.latex(rounded_num_sincos))

('u', 'u', 'u') (0, 0, 0)
('u', 'u', 'x') (0, 0, 0)
('u', 'u', 'y') (0, 0, 0)
('u', 'u', 'z') (0, 0, 0)
('u', 'x', 'u') (0.13*I*sin(0.017*t) - 0.13*I*sin(0.017*t) - 0.25*cos(0.003*t) - 0.25*cos(0.007*t) + 0.25*cos(0.013*t) + 0.13*cos(0.017*t) + 0.13*cos(0.017*t), 0, 0)
('u', 'x', 'x') (0.13*I*sin(0.017*t) - 0.13*I*sin(0.017*t) - 0.25*cos(0.003*t) - 0.25*cos(0.007*t) + 0.25*cos(0.013*t) + 0.13*cos(0.017*t) + 0.13*cos(0.017*t), 0, 0)
('u', 'x', 'y') (0.13*I*sin(0.017*t) - 0.13*I*sin(0.017*t) - 0.25*cos(0.003*t) - 0.25*cos(0.007*t) + 0.25*cos(0.013*t) + 0.13*cos(0.017*t) + 0.13*cos(0.017*t), 0, 0)
('u', 'x', 'z') (0.13*I*sin(0.017*t) - 0.13*I*sin(0.017*t) - 0.25*cos(0.003*t) - 0.25*cos(0.007*t) + 0.25*cos(0.013*t) + 0.13*cos(0.017*t) + 0.13*cos(0.017*t), -0.13*I*(exp(0.003*I*t) - exp(0.007*I*t) - exp(0.013*I*t) + exp(0.017*I*t) - exp(-0.017*I*t) + exp(-0.013*I*t) + exp(-0.007*I*t) - exp(-0.003*I*t)), 0)
('u', 'y', 'u') (0, 0.13*I*sin(0.017*t) - 0.13*I*sin(0.017*t) - 0.25*cos(0.003*t) - 0.